In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data=np.load("C:\\MLDatabases\\google-images-download-master\\downloads\\100x100_resized_images_as_numpy\\x_non_alpha_composed.npy",mmap_mode='r')

In [3]:
labels=np.load("C:\\MLDatabases\\google-images-download-master\\downloads\\100x100_resized_images_as_numpy\\y_non_alpha_composite1.npy",mmap_mode='r')

In [4]:
def create_weights(shape,name,stddev=0.003):
    return tf.Variable(initial_value=tf.truncated_normal(shape=shape, mean=0, stddev=stddev, dtype="float32", seed=10),dtype="float32",name=name)

In [5]:
tf.set_random_seed(10)
np.random.seed(10)

In [6]:
tf.reset_default_graph()
def get_graph():
    
    strides=(1,2,2,1)    
    
    with tf.device("/device:CPU:0"):
    
        x=tf.placeholder(tf.float32,shape=(None,100,100,3),name="x")
        y=tf.placeholder(tf.int64,shape=(None,),name="y")
        lr=tf.placeholder(tf.float32,shape=(),name="lr")
        dr=tf.placeholder(tf.float32,shape=(),name="dr")
        #y_onehot=tf.one_hot(y,2)


        conv_w1=create_weights((10,10,3,2),"conv_w1")
        conv_w2=create_weights((6,6,2,4),"conv_w2")
        conv_w3=create_weights((3,3,4,8),"conv_w3")
        conv_w4=create_weights((2,2,8,8),"conv_w4")
        #conv1_mean=create_weights((16,),"conv1_mean",1)
        #conv2_mean=create_weights((16,),"conv2_mean",1)
        #conv3_mean=create_weights((16,),"conv3_mean",1)
        #conv4_mean=create_weights((16,),"conv4_mean",1)
        #conv1_variance=create_weights((16,),"conv1_variance",1)
        #conv2_variance=create_weights((16,),"conv2_variance",1)
        #conv3_variance=create_weights((16,),"conv3_variance",1)
        #conv4_variance=create_weights((16,),"conv4_variance",1)

        w_fc1=create_weights((7*7*8,512),"w_fc1")
        fc1_bias=create_weights((512,),"fc1_bias")
        w_fc2=create_weights((512,2),"w_fc2")
        fc2_bias=create_weights((2,),"fc2_bias")



        conv1=tf.nn.conv2d(input=x,filter=conv_w1,strides=strides,padding="SAME",name="conv1") #conv1=45
        conv1_relu=tf.nn.relu(conv1,name="relu_conv1")
        conv1_bn=tf.layers.batch_normalization(conv1_relu,training=True)
        #conv1_bn=tf.nn.batch_normalization(x=conv1_relu,mean=conv1_mean,variance=conv1_variance,variance_epsilon=0.000004,offset=None,scale=None)


        conv2=tf.nn.conv2d(input=conv1_bn,filter=conv_w2,strides=strides,padding="SAME",name="conv2") #conv2=21
        conv2_relu=tf.nn.relu(conv2,name="relu_conv2")
        conv2_bn=tf.layers.batch_normalization(conv2_relu,training=True)
        #conv2_bn=tf.nn.batch_normalization(x=conv2_relu,mean=conv2_mean,variance=conv2_variance,variance_epsilon=0.000004,offset=None,scale=None)


        conv3=tf.nn.conv2d(input=conv2_bn,filter=conv_w3,strides=strides,padding="SAME",name="conv3") #conv3=10
        conv3_relu=tf.nn.relu(conv3,name="relu_conv3")
        conv3_bn=tf.layers.batch_normalization(conv3_relu,training=True)
        #conv3_bn=tf.nn.batch_normalization(x=conv3_relu,mean=conv3_mean,variance=conv3_variance,variance_epsilon=0.000004,offset=None,scale=None)


        conv4=tf.nn.conv2d(input=conv3_bn,filter=conv_w4,strides=strides,padding="SAME",name="conv4") #conv4=5
        conv4_relu=tf.nn.relu(conv4,name="relu_conv4")
        conv4_bn=tf.layers.batch_normalization(conv4_relu,training=True)

        #conv4_bn=tf.nn.batch_normalization(x=conv4_relu,mean=conv4_mean,variance=conv4_variance,variance_epsilon=0.000004,offset=None,scale=None)
        flat_feature=tf.reshape(tensor=conv4_bn,shape=[-1,7*7*8],name="flat_feature")
        print(flat_feature)
        fc1=tf.add(tf.matmul(flat_feature,w_fc1),fc1_bias,name="fc1")
        fc1_dropout=tf.nn.dropout(fc1,keep_prob=dr)

        logits=tf.add(tf.matmul(fc1_dropout,w_fc2),fc2_bias,name="fc2")

        with tf.device("/device:CPU:0"):
            softmaxed_logits=tf.nn.softmax(logits,name="softmaxed_logits")    
        loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(None,labels=y, logits=logits),name="loss")

        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        with tf.control_dependencies(update_ops):
            optimizer=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

        accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(softmaxed_logits,axis=1), y),dtype=tf.float32))
    
    return optimizer,loss,accuracy,x,y,lr,dr
    
    
    
    

In [7]:
optimizer,loss,accuracy,x,y,lr,dr=get_graph()

Tensor("flat_feature:0", shape=(?, 392), dtype=float32, device=/device:CPU:0)


In [8]:
init_op=tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
indexes=np.arange(2907)
np.random.shuffle(indexes)
np.random.shuffle(indexes)
np.random.shuffle(indexes)

In [10]:
train=indexes[0:2300]
test=indexes[2300:None]

x_train=data[train]
y_train=labels[train]
x_test=data[test]
y_test=labels[test]

In [11]:
EPOCHS=50
BATCH_SIZE=8
n_samples=2907
with tf.Session() as sess:   
    sess.run(init_op)
    for e in range(EPOCHS):
        start_idx=0
        for s in range(0,2300,BATCH_SIZE):
            x_batch=x_train[s:s+BATCH_SIZE]
            y_batch=y_train[s:s+BATCH_SIZE]
            if s%1000==0:
                _,s_loss,s_accu=sess.run([optimizer,loss,accuracy],feed_dict={x:x_batch,y:y_batch,lr:0.0001,dr:0.4})
                print("Epoch",e,"Iteration",s,"Loss: ",s_loss,"Accuracy: ",s_accu)
            else:
                _,s_loss,s_accu=sess.run([optimizer,loss,accuracy],feed_dict={x:x_batch,y:y_batch,lr:0.0001,dr:0.4})

        """   update the testing results   """
        s_cost,test_accu=sess.run([loss,accuracy],feed_dict={x:x_test,y:y_test,lr:0.001,dr:1.0})
        print("Epoch",e,"Loss: ",s_cost,"Accuracy: ",test_accu)

    """   Save the model   """            
    model_file="C:\\MLDatabases\\google-images-download-master\\downloads\\model"
    saver.save(sess, save_path=model_file)
    

Epoch 0 Iteration 0 Loss:  0.693504 Accuracy:  0.375
Epoch 0 Iteration 1000 Loss:  0.847708 Accuracy:  0.5
Epoch 0 Iteration 2000 Loss:  0.449285 Accuracy:  0.75
Epoch 0 Loss:  0.477726 Accuracy:  0.784185
Epoch 1 Iteration 0 Loss:  0.563366 Accuracy:  0.75
Epoch 1 Iteration 1000 Loss:  0.664143 Accuracy:  0.625
Epoch 1 Iteration 2000 Loss:  0.498785 Accuracy:  0.75
Epoch 1 Loss:  0.454754 Accuracy:  0.808896
Epoch 2 Iteration 0 Loss:  0.54029 Accuracy:  0.625
Epoch 2 Iteration 1000 Loss:  0.594692 Accuracy:  0.625
Epoch 2 Iteration 2000 Loss:  0.274749 Accuracy:  0.875
Epoch 2 Loss:  0.448078 Accuracy:  0.803954
Epoch 3 Iteration 0 Loss:  0.522309 Accuracy:  0.75


KeyboardInterrupt: 

In [3]:
import tensorflow as tf

meta_path = 'C:\\MLDatabases\\google-images-download-master\\downloads\\model.meta' # Your .meta file
output_node_names = ['softmaxed_logits:0']    # Output nodes

with tf.Session() as sess:

    # Restore the graph
    saver = tf.train.import_meta_graph(meta_path)

    # Load weights
    saver.restore(sess,'C:\\MLDatabases\\google-images-download-master\\downloads\\model')

    # Freeze the graph
    frozen_graph_def = tf.graph_util.convert_variables_to_constants(
        sess,
        sess.graph_def,
        output_node_names)

    # Save the frozen graph
    with open('C:\\MLDatabases\\google-images-download-master\\downloads\\model_as_string.pb', 'wb') as f:
      f.write(frozen_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from C:\MLDatabases\google-images-download-master\downloads\model


InvalidArgumentError: Cannot assign a device for operation 'SparseSoftmaxCrossEntropyWithLogits/Shape_1': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/Shape_1 = Shape[T=DT_INT64, out_type=DT_INT32, _device="/device:GPU:0"](y_1:0)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/Shape_1', defined at:
  File "C:\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop